In [1]:
import pandas as pd
import random
import numpy as np
from itertools import combinations
import networkx as nx
from icecream import ic
from geopy.distance import geodesic

In [111]:
# Vanuatu CSV
df = pd.read_csv('./csv/vanuatu.csv', header= None, names=['City', 'x', 'y'])
#print(df.to_string())

In [153]:
# China CSV
df = pd.read_csv('./csv/china.csv', header= None, names=['City', 'x', 'y'])
#print(df.to_string())

In [118]:
# Italy CSV
df = pd.read_csv('./csv/italy.csv', header= None, names=['City', 'x', 'y'])
#print(df.to_string())

In [150]:
# Russia CSV
df = pd.read_csv('./csv/russia.csv', header= None, names=['City', 'x', 'y'])
#print(df.to_string())

In [138]:
# US CSV
df = pd.read_csv('./csv/us.csv', header= None, names=['City', 'x', 'y'])
#print(df.to_string())

# Process CSV and Create Distance Matrix

In [3]:
def compute_distance_matrix():
    cities = df
    distant_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        distant_matrix[c1.Index, c2.Index] = distant_matrix[c2.Index, c1.Index] = round(float(geodesic(
            (c1.x, c1.y), (c2.x, c2.y)).km),2)
    cities.head()
    for i in range(df.shape[0]):
        distant_matrix[i, i] = np.inf
    
    return distant_matrix

In [154]:
distant_matrix = compute_distance_matrix()

In [5]:
def cost(path):
    pay = float(0)
    for i in range(len(path) -1):
        pos_row = path[i]
        pos_col = path[i+1]
        pay += float(distant_matrix[pos_row][pos_col])
        #print(f'going from {pos_row} to {pos_col}') # Uncomment to check if add correct paths
    
    return round(pay, 2)

# Greedy Algorithm
Starting from a random node, move to the nearest node

In [6]:
def greedy_algorithm(starting_point, csv_len, print_path=False):
    matrix = distant_matrix.copy()
    # Randint include both values
    # Initialize all cities as NOT visited
    visited_cities = np.full(csv_len, False)
    focus_point = starting_point
    # Set the city start to True
    visited_cities[focus_point] = True
    if print_path:
        print(f"Starting Point: {focus_point}-{df.iloc[focus_point]['City']}")
    
    n_steps = 0
    # Initialize the path with the starting node.
    route = [focus_point]
        
    # Loop till i "visited" all the cities
    while visited_cities.sum() != csv_len:
        matrix[:, focus_point] = np.inf
        near_city = np.argmin(matrix[focus_point])
        visited_cities[near_city] = True
        route.append(near_city)
        n_steps += 1
        
        if print_path:    
            print(f"i'm going from {focus_point}-{df.iloc[focus_point]['City']} to {near_city}-{df.iloc[near_city]['City']}")
        focus_point = near_city
        

    # Go from last point to the starting point ==> Close the graph
    route.append(starting_point)
    n_steps += 1
    path_cost = cost(route)
    if print_path:
        print(f"i'm going from {near_city}-{df.iloc[near_city]['City']} to {starting_point}-{df.iloc[starting_point]['City']}")
        print(f"n:steps {n_steps}")
        print(f"Route: {route}")
        print(f"Cost: {path_cost}km")
    
    return route, path_cost, n_steps

## First Greedy Algortihm
select a random starting point

In [7]:
csv_len = df.shape[0]
# Randint include both values
starting_point = random.randint(0, csv_len - 1)
_ , _ , _ = greedy_algorithm(starting_point, csv_len, print_path=True)

Starting Point: 533-Tianchang
i'm going from 533-Tianchang to 605-Xinghua
i'm going from 605-Xinghua to 151-Gaoyou
i'm going from 151-Gaoyou to 242-Jiangdu
i'm going from 242-Jiangdu to 634-Yangzhou
i'm going from 634-Yangzhou to 700-Zhenjiang
i'm going from 700-Zhenjiang to 656-Yizheng
i'm going from 656-Yizheng to 292-Jurong
i'm going from 292-Jurong to 403-Nanjing
i'm going from 403-Nanjing to 377-Maanshan
i'm going from 377-Maanshan to 577-Wuhu
i'm going from 577-Wuhu to 58-Chaohu
i'm going from 58-Chaohu to 125-Feidong
i'm going from 125-Feidong to 190-Hefei
i'm going from 190-Hefei to 368-Lujiang
i'm going from 368-Lujiang to 540-Tongcheng
i'm going from 540-Tongcheng to 10-Anqing
i'm going from 10-Anqing to 68-Chizhou
i'm going from 68-Chizhou to 545-Tongling
i'm going from 545-Tongling to 624-Xuanzhou
i'm going from 624-Xuanzhou to 416-Ningguo
i'm going from 416-Ningguo to 654-Yixing
i'm going from 654-Yixing to 354-Liyang
i'm going from 354-Liyang to 280-Jintan
i'm going from 

## Second Greedy Algorithm
Define all segment distance and pick the shortest as starting point


In [8]:
# Create the lis of all possible segment
# having it as a set can be worst instead of using a list
# The overhead of set must be higher
# List of tuple, and in the tuple we have a set and a floating point
csv_len = df.shape[0]
segments = [([c1, c2], float(distant_matrix[c1][c2])) for c1, c2 in combinations(range(csv_len), 2)]
shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))

# Select the first shortest segment and set it as a starting point!
starting_point = shortest[0][0]
csv_len = df.shape[0]
_ , _ , _ = greedy_algorithm(starting_point, csv_len, print_path=True)


Starting Point: 129-Fenyang
i'm going from 129-Fenyang to 369-Lüliang
i'm going from 369-Lüliang to 334-Linfen
i'm going from 334-Linfen to 210-Houma
i'm going from 210-Houma to 673-Yuncheng
i'm going from 673-Yuncheng to 470-Sanmenxia
i'm going from 470-Sanmenxia to 336-Lingbao
i'm going from 336-Lingbao to 661-Yongji
i'm going from 661-Yongji to 180-Hancheng
i'm going from 180-Hancheng to 541-Tongchuan
i'm going from 541-Tongchuan to 563-Weinan
i'm going from 563-Weinan to 587-Xian
i'm going from 587-Xian to 595-Xianyang
i'm going from 595-Xianyang to 607-Xingping
i'm going from 607-Xingping to 22-Baoji
i'm going from 22-Baoji to 185-Hanzhong
i'm going from 185-Hanzhong to 161-Guangyuan
i'm going from 161-Guangyuan to 309-Langzhong
i'm going from 309-Langzhong to 26-Bazhong
i'm going from 26-Bazhong to 90-Daxian
i'm going from 90-Daxian to 92-Dazhou
i'm going from 92-Dazhou to 559-Wanyuan
i'm going from 559-Wanyuan to 7-Ankang
i'm going from 7-Ankang to 476-Shangluo
i'm going from 47

## Third Greedy Algorithm
Loop over all the possible citys as a starting point, save all the cost and select the starting point with lower cost


In [9]:
def print_route(route, path_cost):
    for i in range(len(route) - 1):
        focus_point = route[i]
        near_city = route[i+1]
        print(f"i'm going from {focus_point}-{df.iloc[focus_point]['City']} to {near_city}-{df.iloc[near_city]['City']}")
    print(f"Cost: {path_cost}km")
    

## Run this code to evaluate the population

In [155]:
# Best Timing! For china it took 3s!
results = {"index":0, "route":[], "cost": np.inf}
# Load the population for the next Algorithms!
eval_population = []
csv_len = df.shape[0]
# Total_steps = sum of all steps for each starting point
total_step = 0
# For each starting point 
for i in range(csv_len):
    route, path_cost, n_steps = greedy_algorithm(i, csv_len)
    total_step += n_steps
    eval_population.append({"index":i, "route":route, "cost": path_cost})
    #print(route, path_cost, i) # check
    # Save only if is the shortest path
    if path_cost < results["cost"]:
        results["index"] = i
        results["route"] = route
        results["cost"] = path_cost

print(results)
print(f'Total steps: {total_step}')
print_route(results["route"], results["cost"])
    

{'index': 134, 'route': [134, 393, 554, 49, 495, 302, 555, 297, 521, 39, 652, 550, 1, 2, 209, 473, 298, 15, 299, 551, 179, 115, 672, 257, 287, 691, 266, 579, 25, 704, 438, 584, 338, 646, 89, 502, 176, 571, 575, 341, 24, 120, 671, 629, 180, 661, 673, 470, 336, 645, 290, 386, 637, 598, 156, 94, 614, 621, 679, 48, 625, 371, 574, 422, 467, 609, 699, 293, 617, 229, 562, 187, 14, 344, 568, 182, 474, 608, 402, 291, 198, 493, 612, 147, 365, 496, 374, 613, 103, 6, 21, 458, 193, 41, 44, 219, 143, 529, 183, 534, 308, 217, 31, 548, 636, 469, 508, 52, 387, 123, 715, 146, 27, 530, 345, 723, 61, 339, 59, 34, 139, 33, 417, 337, 282, 233, 604, 443, 556, 429, 79, 710, 142, 87, 649, 286, 12, 326, 95, 518, 491, 670, 108, 138, 539, 537, 615, 328, 36, 126, 81, 105, 542, 18, 342, 211, 418, 383, 327, 512, 157, 46, 288, 93, 676, 569, 507, 284, 263, 251, 114, 196, 358, 635, 549, 228, 413, 173, 395, 396, 197, 289, 101, 450, 394, 232, 23, 506, 239, 191, 398, 643, 538, 515, 174, 28, 572, 411, 449, 231, 357, 680, 6

In [ ]:
# How to search inside a List of Dict
#min(eval_population, key=lambda x:x['cost'])
#max(eval_population, key=lambda x:x['cost'])

{'index': 134,
 'route': [134,
  393,
  554,
  49,
  495,
  302,
  555,
  297,
  521,
  39,
  652,
  550,
  1,
  2,
  209,
  473,
  298,
  15,
  299,
  551,
  179,
  115,
  672,
  257,
  287,
  691,
  266,
  579,
  25,
  704,
  438,
  584,
  338,
  646,
  89,
  502,
  176,
  571,
  575,
  341,
  24,
  120,
  671,
  629,
  180,
  661,
  673,
  470,
  336,
  645,
  290,
  386,
  637,
  598,
  156,
  94,
  614,
  621,
  679,
  48,
  625,
  371,
  574,
  422,
  467,
  609,
  699,
  293,
  617,
  229,
  562,
  187,
  14,
  344,
  568,
  182,
  474,
  608,
  402,
  291,
  198,
  493,
  612,
  147,
  365,
  496,
  374,
  613,
  103,
  6,
  21,
  458,
  193,
  41,
  44,
  219,
  143,
  529,
  183,
  534,
  308,
  217,
  31,
  548,
  636,
  469,
  508,
  52,
  387,
  123,
  715,
  146,
  27,
  530,
  345,
  723,
  61,
  339,
  59,
  34,
  139,
  33,
  417,
  337,
  282,
  233,
  604,
  443,
  556,
  429,
  79,
  710,
  142,
  87,
  649,
  286,
  12,
  326,
  95,
  518,
  491,
  670,
  108,
  13

## Solution With Graph and NetworkX library
Below you can find the implementation of the Greedy solution

In [114]:
import networkx as nx
import matplotlib.pyplot as plt
import random

In [14]:
# In this case the Weights are "inside" the graph, I used the distance matrix only to load the values of the distances

def generate_complete_graph(num_nodes):
    G = nx.complete_graph(num_nodes)
    for u, v in G.edges():
        G.edges[u,v]['weight'] = distant_matrix[u][v]

    return G

def plot_graph_steps(G, tour, current_node, pos):
    # Clear the figure
    plt.clf()
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500)
    path_edges = list(zip(tour, tour[1:]))  # I obtain the path, we can visualize the path
    # print(path_edges)
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)
    nx.draw_networkx_edges(G, pos, nodelist=[current_node], edge_color='green', node_size=500)
    
    edge_labels = nx.get_edge_attributes(G, name='weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    
    # We'll see the construction
    plt.pause(0.5)
    
def calculate_rout_cost(G, tour):
    return sum(G[tour[i]][tour[i+1]]['weight'] for i in range(len(tour) - 1))

def nearest_neightbor_tsp(G, start_node = None, print_graph = False, print_out = True):
    if start_node is None:
        start_node = random.choice(list(G.nodes))
    
    pos = nx.spring_layout(G)
    plt.ion()
    plt.show()
    
    # All nodes are unvisited
    unvisited = set(G.nodes)
    # Remove the starting node
    unvisited.remove(start_node)
    tour = [start_node]
    current_node = start_node
    
    if print_graph:
        plot_graph_steps(G, tour, current_node, pos)
    n_steps = 1
    while unvisited:
        # Closest node to the current one
        next_node = min(unvisited, key=lambda node: G[current_node][node]['weight'])
        # Remove the node
        unvisited.remove(next_node)
        tour.append(next_node)
        current_node = next_node
        n_steps += 1
        if print_graph:
            plot_graph_steps(G, tour, current_node, pos)
    
    tour.append(start_node)
    if print_graph:
        plot_graph_steps(G, tour, start_node, pos)
    tour_cost = calculate_rout_cost(G, tour)
    
    if print_out:
        print(tour)
        print(f'Construnction Heuristic Tour Cost: {tour_cost}')
        print(f'N-steps: {n_steps}')
    
    plt.ioff()
    plt.show()
    return tour, tour_cost, n_steps
    
    

In [226]:
# Generate all the graph edges for a fully connected graph
print(generate_complete_graph(df.shape[0]).edges)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]


## Usage of NetworkX approximation Algorithm
I use this informations for checking if my algorithms produces a good solution

In [156]:
# traveling_salesman_problem Approximation Algorithm
# Find the shortest path in G connecting specified nodes
# Slowest algorithm! It took 2 min for China
# But he got the best results 54456 Km
G = generate_complete_graph(df.shape[0])
approx_tour = nx.approximation.traveling_salesman_problem(G, cycle=True)
approx_tour_cost = calculate_rout_cost(G, approx_tour)
print(approx_tour)
print(approx_tour_cost)

[0, 186, 695, 698, 513, 434, 433, 531, 16, 553, 206, 468, 456, 449, 231, 357, 680, 628, 381, 684, 234, 172, 154, 238, 192, 572, 411, 28, 174, 515, 355, 5, 538, 398, 643, 191, 239, 506, 543, 133, 23, 450, 101, 289, 232, 394, 197, 396, 395, 173, 413, 228, 549, 635, 358, 196, 114, 251, 263, 211, 418, 383, 542, 18, 342, 327, 512, 537, 539, 138, 108, 670, 491, 518, 95, 326, 36, 126, 81, 105, 710, 556, 429, 79, 142, 87, 649, 417, 337, 282, 233, 604, 443, 345, 530, 723, 61, 183, 529, 534, 143, 219, 44, 41, 193, 458, 317, 38, 195, 112, 707, 717, 644, 442, 503, 560, 423, 307, 694, 420, 359, 451, 306, 638, 561, 565, 466, 178, 305, 264, 437, 253, 254, 148, 55, 11, 711, 459, 323, 619, 428, 682, 666, 599, 532, 721, 722, 639, 453, 522, 304, 616, 40, 689, 265, 325, 343, 402, 291, 198, 100, 493, 612, 147, 608, 474, 182, 344, 568, 14, 432, 588, 203, 293, 699, 609, 255, 229, 617, 562, 187, 349, 56, 149, 267, 444, 386, 637, 598, 156, 614, 621, 679, 422, 574, 714, 618, 160, 8, 517, 647, 596, 181, 594, 535

In [147]:
# Simulated annealing TSP, NetworkX Algorithm
# Returns an approximate solution to the TSP
# Fastest algorithm! It took 0.5s for China
G = generate_complete_graph(df.shape[0])
# Instead of "greedy" you can pass directly the starting path as a list [0, 3, 5, 6, 2, 4, 1, 7, 0] EX. approx_tour
annealing_tour = nx.approximation.simulated_annealing_tsp(G, "greedy")
print(annealing_tour)
print(cost(annealing_tour))

[0, 321, 72, 154, 40, 225, 4, 168, 103, 240, 107, 173, 66, 133, 110, 100, 311, 141, 249, 18, 257, 64, 79, 167, 34, 148, 286, 219, 151, 126, 296, 57, 304, 273, 22, 143, 21, 174, 193, 181, 183, 60, 164, 15, 261, 14, 16, 270, 198, 45, 322, 94, 41, 234, 77, 114, 122, 323, 61, 46, 245, 3, 12, 313, 59, 20, 222, 5, 80, 83, 190, 136, 195, 199, 213, 283, 325, 218, 235, 31, 192, 314, 117, 324, 161, 37, 28, 233, 32, 230, 191, 163, 166, 290, 7, 35, 55, 1, 224, 299, 10, 71, 75, 312, 284, 147, 111, 276, 99, 70, 52, 155, 160, 90, 53, 189, 185, 128, 47, 142, 26, 169, 157, 134, 292, 105, 135, 210, 144, 30, 244, 293, 253, 54, 281, 214, 229, 318, 227, 98, 124, 176, 180, 220, 69, 62, 120, 175, 153, 38, 48, 309, 196, 116, 194, 130, 137, 188, 17, 82, 49, 247, 165, 178, 113, 246, 252, 179, 275, 207, 156, 300, 206, 211, 139, 129, 152, 320, 303, 33, 197, 205, 6, 162, 177, 204, 86, 149, 2, 242, 232, 58, 121, 43, 73, 297, 317, 13, 145, 29, 112, 97, 231, 316, 319, 256, 27, 171, 187, 279, 278, 23, 238, 140, 95, 29

In [146]:
# Christofides, NetworkX Algorithm
# Seems to find the same solution as simulated Annealing
# Compute a 3/2-approximation of the traveling salesman problem in a complete undirected graph using Christofides [1] algorithm.
# Acceptable Algorithm! It took 12.5s for China
christofides_tour = nx.approximation.christofides(G)
print(annealing_tour)
print(cost(annealing_tour))

[0, 321, 72, 154, 40, 225, 4, 168, 103, 240, 107, 173, 66, 133, 110, 100, 311, 141, 249, 18, 257, 64, 79, 167, 34, 148, 286, 219, 151, 126, 296, 57, 304, 273, 22, 143, 21, 174, 193, 181, 183, 60, 164, 15, 261, 14, 16, 270, 198, 45, 322, 94, 41, 234, 77, 114, 122, 323, 61, 46, 245, 3, 12, 313, 59, 20, 222, 5, 80, 83, 190, 136, 195, 199, 213, 283, 325, 218, 235, 31, 192, 314, 117, 324, 161, 37, 28, 233, 32, 230, 191, 163, 166, 290, 7, 35, 55, 1, 224, 299, 10, 71, 75, 312, 284, 147, 111, 276, 99, 70, 52, 155, 160, 90, 53, 189, 185, 128, 47, 142, 26, 169, 157, 134, 292, 105, 135, 210, 144, 30, 244, 293, 253, 54, 281, 214, 229, 318, 227, 98, 124, 176, 180, 220, 69, 62, 120, 175, 153, 38, 48, 309, 196, 116, 194, 130, 137, 188, 17, 82, 49, 247, 165, 178, 113, 246, 252, 179, 275, 207, 156, 300, 206, 211, 139, 129, 152, 320, 303, 33, 197, 205, 6, 162, 177, 204, 86, 149, 2, 242, 232, 58, 121, 43, 73, 297, 317, 13, 145, 29, 112, 97, 231, 316, 319, 256, 27, 171, 187, 279, 278, 23, 238, 140, 95, 29

## My Greedy solution with Graph


In [21]:

# Second Fastest Algorithm! It took 5s for China
# The solution without using the Graph it take about 0.3s
G = generate_complete_graph(df.shape[0])
#print(generate_complete_graph(df.shape[0]).edges)
# Set to True only if the problem has few edges, otherwise the visualizzation will be chaotyc
# Start from a Random node ==> If start_node == None ==> set random starting node
_ , _ , _ = nearest_neightbor_tsp(G, start_node=None, print_graph=False)

[698, 513, 434, 433, 531, 16, 553, 357, 231, 449, 456, 468, 355, 206, 5, 695, 186, 0, 504, 676, 569, 507, 284, 263, 288, 93, 46, 157, 512, 327, 383, 418, 211, 251, 114, 196, 358, 635, 549, 228, 413, 173, 395, 396, 197, 289, 101, 450, 394, 232, 23, 506, 239, 191, 398, 643, 538, 515, 174, 28, 572, 411, 680, 628, 172, 234, 684, 381, 154, 238, 192, 543, 133, 480, 18, 342, 542, 138, 108, 670, 491, 518, 95, 326, 12, 286, 328, 615, 537, 539, 36, 126, 81, 105, 710, 556, 429, 79, 420, 359, 694, 451, 306, 305, 423, 307, 55, 560, 11, 148, 254, 437, 264, 253, 711, 459, 323, 619, 428, 519, 216, 212, 533, 605, 151, 242, 634, 700, 656, 292, 403, 377, 577, 58, 125, 190, 368, 540, 10, 68, 545, 624, 416, 654, 354, 280, 675, 85, 526, 272, 247, 685, 407, 54, 301, 523, 475, 424, 256, 547, 177, 578, 237, 97, 184, 140, 331, 713, 653, 111, 662, 275, 310, 241, 455, 245, 478, 98, 319, 269, 194, 399, 145, 690, 620, 240, 270, 719, 64, 315, 51, 436, 199, 592, 484, 591, 716, 330, 589, 427, 352, 53, 391, 669, 347, 2

In [239]:
# For China it takes too much time!
# We have 725 node, and since it takes approximately 5s to find the solution for one node
# It should take about 725 * 5 / 60 = about 60 min
# The solution with Graph is much slower compared to the First greedy solution!
results = {"index":0, "route":[], "cost": np.inf}
csv_len = df.shape[0]
G = generate_complete_graph(csv_len)
# Total_steps = sum of all steps for each starting point
total_step = 0
# For each starting point 
for i in range(csv_len):
    route, path_cost, n_steps = nearest_neightbor_tsp(G, start_node=i, print_graph=False, print_out=False)
    total_step += n_steps
    #print(route, path_cost, i) # check
    # Save only if is the shortest path
    if path_cost < results["cost"]:
        results["index"] = i
        results["route"] = route
        results["cost"] = path_cost

print(results)
print(f'Total steps: {total_step}')
print_route(results["route"], results["cost"])

{'index': 0, 'route': [0, 7, 1, 4, 3, 5, 2, 6, 0], 'cost': 1475.53}
Total steps: 64
i'm going from 0-Isangel to 7-Vila
i'm going from 7-Vila to 1-Lakatoro
i'm going from 1-Lakatoro to 4-Norsup
i'm going from 4-Norsup to 3-Luganville
i'm going from 3-Luganville to 5-Port Olry
i'm going from 5-Port Olry to 2-Longana
i'm going from 2-Longana to 6-Sola
i'm going from 6-Sola to 0-Isangel
Cost: 1475.53km


## First Genetic Algorithm!
Genetic algorithm are called Meta-Heuristic, they are used to solve Optimization problems using an Evolutionary Approach

In [78]:
from dataclasses import dataclass

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

In [108]:
POPULATION_SIZE = len(eval_population) # Starting solutions retrieved by Greedy approach
GENOME_LENGTH = len(eval_population[0]["route"]) # Lenght of the solution
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.7 # I have 70% of doing crossover otherwise i return just the parents without any crossover
GENERATIONS = 200 # Whole process repeted 200 times

## CROSSOVER FUNCTIONS

In [93]:
def one_point_crossover(parent1, parent2):
    # I don't do always the crossover
    if random.random() < CROSSOVER_RATE:
        crossover_point = random.randint(1, GENOME_LENGTH - 1)
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]
    else:
        return parent1, parent2
    
def two_point_crossover(parent1, parent2):
    # I don't do always the crossover
    if random.random() < CROSSOVER_RATE:
        cp1 = random.randint(0, GENOME_LENGTH - 1)
        cp2 = random.randint(cp1 + 1, GENOME_LENGTH - 1)
        # Exchange the middle part
        chd_1 = parent1[:cp1] + parent2[cp1:cp2] + parent1[cp2:]
        chd_2 = parent2[:cp1] + parent1[cp1:cp2] + parent2[cp2:]
        return chd_1 , chd_2 
    else:
        return parent1, parent2
    
def uniform_crossover(parents):
    # For every locus pick the genenfrom a random parent
    new_genome = []
    for i in range(GENOME_LENGTH):
        rand_parent = random.randint(0,len(parents))
        new_genome.append(parents[rand_parent][i])
    return new_genome

def Cycle_crossover(parent1, parent2):
    cp1 = random.randint(0, GENOME_LENGTH - 1)
    if cp1 == GENOME_LENGTH - 1:
        cp2 = cp1
        cp1 = cp1 - 1
    else:
        cp2 = random.randint(cp1 + 1, GENOME_LENGTH - 1)
    middle_genome= parent1[cp1:cp2]

    init_genome = []
    for i in range(cp1 - 1):
        find_spot = False
        j = 0
        while not find_spot and j < GENOME_LENGTH:
            if parent2[j] not in middle_genome and parent2[j] not in init_genome:
                find_spot = True
                init_genome.append(parent2[j])
            j += 1

    end_genome = []
    for i in range(cp2, GENOME_LENGTH):
        find_spot = False
        j = 0
        while not find_spot and j < GENOME_LENGTH:
            if parent2[j] not in middle_genome and parent2[j] not in init_genome and parent2[j] not in end_genome:
                find_spot = True
                end_genome.append(parent2[j])
            j += 1
    #print(init_genome, middle_genome, end_genome)
    new_genome = init_genome + middle_genome + end_genome
    return new_genome

def Partially_mapped_crossover(parent1, parent2):
    cp1 = random.randint(0, GENOME_LENGTH - 1)
    if cp1 == GENOME_LENGTH - 1:
        cp2 = cp1
        cp1 = cp1 - 1
    else:
        cp2 = random.randint(cp1 + 1, GENOME_LENGTH - 1)
    middle_genome= parent1[cp1:cp2]

    new_genome = []
    for i in range(GENOME_LENGTH):
        if i >= cp1 and i < cp2:
            new_genome.append(middle_genome[i - cp1])
        else:
            index = parent2.index(parent1[i])
            elem = parent1[index]
            if elem in middle_genome:
                elem = parent1[i]
            new_genome.append(elem)         

    return new_genome

def Inver_over(parent1, parent2):
    # Select one random locus from P1, remove last element to avoid pick last element
    cp1 = random.randint(0, GENOME_LENGTH - 2)
    element_1 = parent1[cp1]

    # Find where is the selected element in the second parent
    index_in_parent_2 = parent2.index(element_1)
    # Check the position of the item, and select his neighbor
    # If the item is in the last loci take the previous neightbor
    # Take next loci value
    select_item_parent_2 = index_in_parent_2 + 1
    take_previous = False
    # If is last take previous loci value
    if index_in_parent_2 == GENOME_LENGTH - 1:
        select_item_parent_2 = index_in_parent_2 - 1
        take_previous = True
    element_2 = parent2[select_item_parent_2]
    # Find index in first parent
    cp2 = parent1.index(element_2)

    if take_previous:
        # swap index in order to have all straight forward
        aus = cp1
        cp1 = cp2 
        cp2 = aus
        # Swap values
        aus = element_1
        element_1 = element_2
        element_2 = aus

    print(f'element1: {element_1} pos: {cp1}')
    print(f'element2: {element_2} pos: {cp2}')
    print(f'Take previous : {take_previous}')

    #print(parent1)
    new_genome = parent1.copy()[:cp1+2]
    #print(new_genome)
    value_to_save = new_genome[cp1+1]
    new_genome[cp1+1] = element_2
    last_genome = parent1.copy()[cp1+1:]
    # Remove first item
    last_genome.pop(0)
    if cp2 < cp1:
        new_genome[cp2] = value_to_save
    else:
        index_eleme = last_genome.index(element_2)
        last_genome[index_eleme] = value_to_save
    reverse_last_genome = last_genome[::-1]
    #print(last_genome[::-1])
    #print(value_to_save)
    #print(new_genome)
    new_genome = new_genome + last_genome
    return new_genome

# Taken from GIORGIO BONGIOVANNI
def inver_over(individual, population, p=0.02):
    # Copy of the individual's genome to prevent direct modifications
    new_genome = individual.genome.copy()
    original_fitness = individual.fitness  # Save the original fitness for comparison
    
    # Select a random city as the starting point
    current_city = random.choice(new_genome)
    
    while True:
        # Decide how to select the city `c'`
        if random.random() < p:  # Internal mutation
            # Select a random remaining city from the genome
            next_city = random.choice([city for city in new_genome if city != current_city])
        else:  # External crossing
            # Select another random individual from the population
            other_individual = random.choice(population).genome
            current_index = list(other_individual).index(current_city)
            # `next_city` is the city following `current_city` in the second individual
            next_city = other_individual[(current_index + 1) % len(other_individual)]
        
        # Find the positions of `current_city` and `next_city` in the current genome
        current_index = list(new_genome).index(current_city)
        next_index = list(new_genome).index(next_city)
        
        # Check if `current_city` and `next_city` are already adjacent
        if abs(current_index - next_index) == 1:
            break  # Exit the loop if they are already adjacent
        
        # Reverse the subsequence between `current_city` and `next_city`
        if current_index > next_index:
            current_index, next_index = next_index, current_index
        new_genome[current_index + 1:next_index + 1] = new_genome[current_index + 1:next_index + 1][::-1]
        
        # Update `current_city` to `next_city` to continue the loop
        current_city = next_city

    # Calculate the fitness of the new solution
    new_fitness = -cost(new_genome)

    # Replace the individual in the population if the fitness has improved
    if new_fitness > original_fitness:
        # Directly update the fields of the individual in the population
        individual.genome = new_genome
        individual.fitness = new_fitness

## PARENT SELECTION FUNCTION

In [ ]:
def select_parent(population, fitness_values):
    total_fitness = sum(fitness_values)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness_value in zip(population, fitness_values):
        current += fitness_value
        if current > pick:
            return individual

def mutate(genome):
    for i in range(len(genome)):
        if random.random() < MUTATION_RATE:
            genome[i] = not genome[i]
    return genome

def tournament_selection(population, torunament_size):
    tournament_individuals = []
    for i in range(tournament_selection):
        random_id = int(random() * population.size())
        tournament_individuals.append(population[random_id])
    
    return tournament_individuals
    

In [95]:
def populate_population(population):
    new_population = []
    for individual in population:
        new_individual = Individual(genome=np.array(individual["route"]), fitness=-individual["cost"])
        new_population.append(new_individual)
    return new_population


In [96]:
populate_population(eval_population)

[Individual(genome=array([  0, 186, 504, 676, 569, 507, 284, 263, 288,  93,  46, 157, 512,
        327, 383, 418, 211, 251, 114, 196, 358, 635, 549, 228, 413, 173,
        395, 396, 197, 289, 101, 450, 394, 232,  23, 506, 239, 191, 398,
        643, 538, 515, 174,  28, 572, 411, 449, 231, 357, 680, 628, 172,
        234, 684, 381, 154, 238, 192, 456, 468, 355, 206,   5, 695, 698,
        513, 434, 433, 531,  16, 553, 544, 505, 537, 539, 138, 108, 670,
        491, 518,  95, 326,  12, 286, 328, 615,  33, 139,  34,  59, 282,
        337, 417, 649,  87, 142, 556, 429,  79, 710, 105,  81, 126,  36,
        542,  18, 342, 480, 133, 543, 233, 604, 443, 345, 530, 183, 529,
        143, 534, 308, 217,  31, 548, 636, 469, 508,  52, 387, 123, 715,
        146,  27, 458, 193,  41,  44, 219, 317,  38, 112, 195, 503, 442,
        644, 717, 707, 689,  40, 304, 616, 522, 265, 325, 343, 402, 291,
        198, 493, 612, 147, 365, 496, 374, 613, 103,   6,  21, 100, 608,
        474, 182, 344, 568,  14, 

In [109]:
def evolutionary_algorithm_tsp():
    population = populate_population(eval_population)

    for generation in range(GENERATIONS):
        for individual in population:
            # Create a new version of the individual using inver-over
            inver_over(individual, population, p=0.2)

        # Print the best individual of the generation
        best_individual = max(population, key=lambda individual: individual.fitness)
        print(f"Generation {generation} - Best fitness: {-best_individual.fitness}")
    
    # Return the best solution found
    best_individual = max(population, key=lambda individual: individual.fitness)
    print("Best path found:", best_individual.genome)
    print("Distance of the best path:", -best_individual.fitness)
    return best_individual

In [157]:
evolutionary_algorithm_tsp()

Generation 0 - Best fitness: 61660.71
Generation 1 - Best fitness: 61660.71
Generation 2 - Best fitness: 61660.71
Generation 3 - Best fitness: 61660.71
Generation 4 - Best fitness: 61660.71
Generation 5 - Best fitness: 61660.71
Generation 6 - Best fitness: 61660.71
Generation 7 - Best fitness: 61660.71
Generation 8 - Best fitness: 61660.71
Generation 9 - Best fitness: 61660.71
Generation 10 - Best fitness: 61660.71
Generation 11 - Best fitness: 61660.71
Generation 12 - Best fitness: 61660.71
Generation 13 - Best fitness: 61660.71
Generation 14 - Best fitness: 61660.71
Generation 15 - Best fitness: 61660.71
Generation 16 - Best fitness: 61660.71
Generation 17 - Best fitness: 61660.71
Generation 18 - Best fitness: 61660.71
Generation 19 - Best fitness: 61660.71
Generation 20 - Best fitness: 61660.71
Generation 21 - Best fitness: 61660.71
Generation 22 - Best fitness: 61660.71
Generation 23 - Best fitness: 61660.71
Generation 24 - Best fitness: 61660.71
Generation 25 - Best fitness: 61660

Individual(genome=array([569, 676, 507, 284, 263, 288,  93,  46, 157, 512, 327, 383, 418,
       211, 251, 114, 196, 358, 635, 549, 228, 413, 173, 395, 396, 197,
       289, 101, 450, 394, 232,  23, 506, 239, 191, 398, 643, 538, 515,
       174,  28, 572, 411, 449, 231, 357, 680, 628, 172, 234, 684, 381,
       154, 238, 192, 456, 468, 355, 206,   5, 695, 186,   0, 504, 698,
       513, 434, 433, 531,  16, 553, 544, 505, 537, 539, 138, 108, 670,
       491, 518,  95, 326,  12, 286, 328, 615,  33, 139,  34,  59, 282,
       337, 417, 649,  87, 142, 556, 429,  79, 710, 105,  81, 126,  36,
       542,  18, 342, 480, 133, 543, 233, 604, 443, 345, 530, 183, 529,
       143, 534, 308, 217,  31, 548, 636, 469, 508,  52, 387, 123, 715,
       146,  27, 458, 193,  41,  44, 219, 317,  38, 112, 195, 503, 442,
       644, 717, 707, 689,  40, 304, 616, 522, 265, 325, 343, 402, 291,
       198, 493, 612, 147, 365, 496, 374, 613, 103,   6,  21, 100, 608,
       474, 182, 344, 568,  14, 187, 349, 229,